In [42]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

# Load JSON files and store the raw data in some variable. Edit as appropriate
def access_covid_api(filters, structure):
    api = Cov19API(filters, structure)
    data = api.get_json()
    with open('covidJson.json', 'wt') as OUTF:
        json.dump(data, OUTF)
    with open('covidJson.json', 'rt') as INFILE:
        covidData = json.load(INFILE)
    return covidData

filters = ['areaType=overview']

# values here are the names
structure = {"date": "date", 
             "One":"cumPeopleVaccinatedFirstDoseByPublishDate", 
             "Two":"cumPeopleVaccinatedSecondDoseByPublishDate"
            }

#if the test is running correctly
covidData = access_covid_api(filters, structure)

def parse_date(date):
    return pd.to_datetime(date, format="%Y-%m-%d")

def wrangle_data(oldList):
    covidList = oldList['data']
    dates = [dic['date'] for dic in covidList]
    dates.sort()
    startDate = parse_date(dates[0])
    endDate = parse_date(dates[-1])
    index = pd.date_range(startDate, endDate, freq = 'D')
    covidJsondf = pd.DataFrame(index = index, columns = ['One', 'Two'])
    for line in covidList:
        data = parse_date(line['date'])
        for column in ['One', 'Two']:
            value = float(line[column] if line[column] != None else 0.0)
            covidJsondf.loc[data, column] = value                    
    covidJsondf.fillna(0.0, inplace = True)
    return covidJsondf
                          
covidJsondf = wrangle_data(covidData)

covidClos = wdg.SelectMultiple(options = ['One', 'Two'], 
                               value = ['One', 'Two'], 
                               rows = 3, 
                               description = 'Vaccinated:', 
                               disabled = False
                              )

scale = wdg.RadioButtons(options = ['Linear', 'Log'], 
                         description = 'Scale:', 
                         disabled = False
                        )

#Use the VBox to replace the HBox
controls = wdg.VBox([covidClos, scale])

def covid_graph(gCols, gScale):
    if gScale == 'Linear':
        logScale = False
    elif gScale == 'Log':
        logScale = True
    ncols = len(gCols)
    if ncols > 0:
        covidJsondf[list(gCols)].plot(logy = logScale)
        plt.show() # important! update won't work properly without this
    else:
        print("Click to select data for graph")
        print("CTRL-Click to select more than one category)")
        
# keep calling timeseries_graph(gCols=value_of_series, gScale=value_of_scale);
# capture output in widget graph
graph = wdg.interactive_output(covid_graph, {'gCols' : covidClos, 'gScale': scale})

display(controls, graph)

# Get fresh data from the API
def api_button_callback(button):
    covidData = access_covid_api(filters, structure)
    global covidJsondf
    covidJsondf = wrangleData(covidData)
    apiButton.icon = "check"
    apiButton.disabled = True
    
apiButton = wdg.Button(description = 'REFRESH', 
                       disabled = False, 
                       button_style = 'info', # 'success', 'info', 'warning', 'danger' or ''
                       toolTip = "Update vaccination data",
                      )

# remember to register the button callback function with the button    
apiButton.on_click(api_button_callback)# the name of the function inside these brackets

display(apiButton)

Output()

Button(button_style='info', description='REFRESH', style=ButtonStyle())